# Data preparation and feature engineering

Purpose:
- Take the raw benchmark file and apply minimal structural cleaning.
- Engineer business-friendly features for frequency modeling (GLM + XGBoost).
- Lump rare VehBrand / Region levels into a stable `Other` category.
- Export a single, consistent modeling table `claims_prepared.csv`.

# 1. Imports and configuration

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

sns.set(style="whitegrid")

DATA_PATH = "../data/freMTPL2freq.csv"

Raw shape: (678013, 12)


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


# 2. Load raw data

In [2]:
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
df_raw.head()

Total missing values: 0
Number of rows with non-positive Exposure: 0
Number of rows with negative ClaimNb: 0
Cleaned shape: (678013, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678013 non-null  int64   
 1   ClaimNb     678013 non-null  int64   
 2   Exposure    678013 non-null  float64 
 3   Area        678013 non-null  category
 4   VehPower    678013 non-null  int64   
 5   VehAge      678013 non-null  int64   
 6   DrivAge     678013 non-null  int64   
 7   BonusMalus  678013 non-null  int64   
 8   VehBrand    678013 non-null  category
 9   VehGas      678013 non-null  category
 10  Density     678013 non-null  int64   
 11  Region      678013 non-null  category
dtypes: category(4), float64(1), int64(7)
memory usage: 44.0 MB


# 3. Basic cleaning and structural sanity checks
 
Business rules enforced here:
- `IDpol` is treated as a pure identifier (integer key).
- `Exposure` must be strictly positive for frequency modeling.
- `ClaimNb` must not be negative.
- This benchmark dataset is expected to have no missing values; if any appear, this is treated as a data quality issue.

In [3]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Minimal structural cleaning for the frequency modeling benchmark:
    - enforce IDpol type,
    - cast core categoricals,
    - drop rows with non-positive exposure,
    - assert no negative ClaimNb and no missing values.
    """
    df = df.copy()

    # Enforce integer key for policy ID.
    df["IDpol"] = df["IDpol"].astype(int)

    # Core rating factors treated as categorical.
    for col in ["Area", "VehBrand", "VehGas", "Region"]:
        df[col] = df[col].astype("category")

    # Missing values: if any appear, fail fast rather than silently imputing.
    missing_total = df.isna().sum().sum()
    print(f"Total missing values: {missing_total}")
    if missing_total > 0:
        raise ValueError("Unexpected missing values in the dataset.")

    # Exposure must be strictly positive to make sense for frequency modeling.
    invalid_exposure = (df["Exposure"] <= 0).sum()
    print(f"Number of rows with non-positive Exposure: {invalid_exposure}")
    df = df[df["Exposure"] > 0].copy()

    # Negative claim counts would indicate corrupted data.
    invalid_claims = (df["ClaimNb"] < 0).sum()
    print(f"Number of rows with negative ClaimNb: {invalid_claims}")
    if invalid_claims > 0:
        raise ValueError("Unexpected negative ClaimNb values.")

    return df


df_clean = clean_data(df_raw)

print("Cleaned shape:", df_clean.shape)
df_clean.info()

# 4. Feature engineering for frequency modeling

Design choices:
- `ClaimFreq = ClaimNb / Exposure`: kept mainly for EDA and diagnostics; the modeling target remains `ClaimNb` with `Exposure` as a weight.
- `DrivAgeBand`: broad, business-friendly driver age groups aligned with the EDA histograms.
- `VehAgeBand`: new / standard used / older / very old vehicles, based on volume patterns.
- `BonusMalusBand`: clear separation between strong bonus, neutral and strong malus.
- `logDensity`: log-transform of population density to stabilize the heavy right tail.

In [4]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create engineered features used across GLM and XGBoost notebooks:
    - Claim frequency (for diagnostics),
    - driver age bands,
    - vehicle age bands,
    - bonus-malus bands,
    - log-transformed population density.
    """
    df = df.copy()

    # Claim frequency is central for EDA and segment diagnostics.
    df["ClaimFreq"] = df["ClaimNb"] / df["Exposure"]

    # Driver age bands:
    # young (<25), early mid (25–29), core (30–44), mature (45–59), seniors (60+).
    df["DrivAgeBand"] = pd.cut(
        df["DrivAge"],
        bins=[18, 25, 30, 45, 60, 101],
        labels=["18-24", "25-29", "30-44", "45-59", "60+"],
        right=False,
    )

    # Vehicle age bands:
    # 0–2: new cars; 3–7: main mass of used cars; 8–15: older but relevant;
    # 16+: very old, thin portfolio tail.
    df["VehAgeBand"] = pd.cut(
        df["VehAge"],
        bins=[0, 3, 8, 16, 200],
        labels=["0-2", "3-7", "8-15", "16+"],
        right=False,
    )

    # Bonus-Malus bands:
    # strong bonus, moderate bonus, neutral, mild malus, strong malus.
    df["BonusMalusBand"] = pd.cut(
        df["BonusMalus"],
        bins=[50, 60, 80, 100, 130, 300],
        labels=["50-59", "60-79", "80-99", "100-129", "130+"],
        right=False,
    )

    # Log-transform of density to capture diminishing marginal effects
    # in very dense areas.
    df["logDensity"] = np.log1p(df["Density"])

    # Ensure engineered bands are categorical.
    for col in ["DrivAgeBand", "VehAgeBand", "BonusMalusBand"]:
        df[col] = df[col].astype("category")

    return df


df_prepared = engineer_features(df_clean)
df_prepared.head()

Total missing values: 0
Number of rows with non-positive Exposure: 0
Number of rows with negative ClaimNb: 0
Cleaned shape: (678013, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678013 non-null  int64   
 1   ClaimNb     678013 non-null  int64   
 2   Exposure    678013 non-null  float64 
 3   Area        678013 non-null  category
 4   VehPower    678013 non-null  int64   
 5   VehAge      678013 non-null  int64   
 6   DrivAge     678013 non-null  int64   
 7   BonusMalus  678013 non-null  int64   
 8   VehBrand    678013 non-null  category
 9   VehGas      678013 non-null  category
 10  Density     678013 non-null  int64   
 11  Region      678013 non-null  category
dtypes: category(4), float64(1), int64(7)
memory usage: 44.0 MB


,VehBrand,VehBrand_lumped,Region,Region_lumped
0,B12,B12,R82,R82
1,B12,B12,R82,R82
2,B12,B12,R22,Other
3,B12,B12,R72,R72
4,B12,B12,R72,R72


# 5. Lumping rare VehBrand and Region levels

Motivation (from EDA notebook):
- Very small categories (e.g. niche car brands or small regions) introduce unstable GLM parameters and noisy tree splits.
- Lumping them into an `Other` bucket stabilizes the model without losing material business signal.

Thresholds reused from EDA:
- `VehBrand`: keep brands with at least **20k** policies (≈7% of the portfolio in the rare tail).
- `Region`: keep regions with at least **12k** policies (≈9% of the portfolio in the rare tail).

In [9]:
def lump_rare_categories(
    df: pd.DataFrame,
    col: str,
    min_count: int,
    other_label: str = "Other",
) -> pd.Series:
    """
    Replace rare categories in `col` by a shared `other_label`, based on a
    minimum number of policies per level.
    """
    counts = df[col].value_counts()
    rare = counts[counts < min_count].index

    # Work on an object series to avoid categorical alignment issues.
    s = df[col].astype("object").copy()
    s.loc[s.isin(rare)] = other_label

    return s.astype("category")


# VehBrand: keep brands with at least 20k policies, others go to 'Other'.
df_prepared["VehBrand_lumped"] = lump_rare_categories(
    df_prepared,
    col="VehBrand",
    min_count=20000,
)

# Region: keep regions with at least 12k policies, others go to 'Other'.
df_prepared["Region_lumped"] = lump_rare_categories(
    df_prepared,
    col="Region",
    min_count=12000,
)

df_prepared[["VehBrand", "VehBrand_lumped", "Region", "Region_lumped"]].head()

C:\Users\shomi\AppData\Local\Temp\ipykernel_33576\860296220.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("VehBrand_lumped")


,policies,exposure,claims,crash_rate,claim_freq
VehBrand_lumped,,,,,
B12,166024,64808.608718,8859,0.049728,0.136695
B5,34753,19992.062695,2020,0.055103,0.101040
B3,53395,28592.876331,2818,0.050023,0.098556
B4,25179,13775.496080,1312,0.049803,0.095242
Other,47517,25430.331361,2394,0.047793,0.094140
B6,28548,15684.953599,1462,0.048620,0.093210
B1,162736,95351.059578,8677,0.050247,0.091001
B2,159861,94864.057101,8560,0.050844,0.090234



Row for B12:


,policies,exposure,claims,crash_rate,claim_freq
VehBrand_lumped,,,,,
B12,166024,64808.608718,8859,0.049728,0.136695


# 6. Brand-level sanity check (focus on B12)

Why this check is useful:
- XGBoost feature importance later shows `VehBrand_lumped` as a strong driver.
- This table confirms that the dominant brand (B12) indeed carries a higher claim frequency than many other brands, which explains the importance.
- `crash_rate` compares the share of policies with at least one claim, while `claim_freq` looks at claims per exposure-year.

In [5]:
df_eda = df_prepared.copy()

# Binary indicator: at least one claim during the exposure period.
df_eda["has_claim"] = (df_eda["ClaimNb"] > 0).astype(int)

brand_summary = (
    df_eda
    .groupby("VehBrand_lumped")
    .agg(
        policies=("IDpol", "count"),
        exposure=("Exposure", "sum"),
        claims=("ClaimNb", "sum"),
        crash_rate=("has_claim", "mean"),
    )
)

brand_summary["claim_freq"] = brand_summary["claims"] / brand_summary["exposure"]

brand_summary_sorted = brand_summary.sort_values("claim_freq", ascending=False)
display(brand_summary_sorted)

print("\nRow for B12:")
if "B12" in brand_summary_sorted.index:
    display(brand_summary_sorted.loc[["B12"]])
else:
    print("B12 not present as a separate level after lumping.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   ClaimNb          678013 non-null  int64   
 1   Exposure         678013 non-null  float64 
 2   IDpol            678013 non-null  int64   
 3   Area             678013 non-null  category
 4   VehBrand_lumped  678013 non-null  category
 5   VehGas           678013 non-null  category
 6   Region_lumped    678013 non-null  category
 7   VehPower         678013 non-null  int64   
 8   VehAge           678013 non-null  int64   
 9   DrivAge          678013 non-null  int64   
 10  BonusMalus       678013 non-null  int64   
 11  Density          678013 non-null  int64   
 12  logDensity       678013 non-null  float64 
 13  DrivAgeBand      678013 non-null  category
 14  VehAgeBand       678013 non-null  category
 15  BonusMalusBand   678013 non-null  category
dtypes: category(7), floa

# 7. Final modeling table and export

The downstream modeling notebooks (GLM, XGBoost) expect a single table:
- one row per policy-period,
- raw target (`ClaimNb`) and `Exposure`,
- policy identifier (`IDpol`),
- a consistent set of engineered and lumped features.

In [ ]:
target_col = "ClaimNb"
exposure_col = "Exposure"

# Choose which brand/region versions to use for modeling.
brand_col = "VehBrand_lumped"   # could be swapped to 'VehBrand' if needed
region_col = "Region_lumped"    # could be swapped to 'Region'

feature_cols = [
    "Area",
    brand_col,
    "VehGas",
    region_col,
    "VehPower",
    "VehAge",
    "DrivAge",
    "BonusMalus",
    "Density",
    "logDensity",
    "DrivAgeBand",
    "VehAgeBand",
    "BonusMalusBand",
]

# Final modeling table: ClaimNb, Exposure, IDpol + all selected features.
df_model = df_prepared[[target_col, exposure_col, "IDpol"] + feature_cols].copy()

print("Modeling table info:")
df_model.info()
df_model.head()

df_model.to_csv("../data/claims_prepared.csv", index=False)
print("\nSaved prepared dataset to ../data/claims_prepared.csv")